IMPORT THE LIBRARIES

In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

PART-1:- DATA PREPROCESSING

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,                   # Normalize pixel values [0, 255] → [0, 1]
    rotation_range=20,               # Random rotation
    width_shift_range=0.1,           # Horizontal translation
    height_shift_range=0.1,          # Vertical translation
    shear_range=0.1,                 # Shearing
    zoom_range=0.1,                  # Zoom in/out
    horizontal_flip=True,            # Flip image horizontally
    fill_mode='nearest'              # Fill in pixels after transformation
)


In [8]:
val_datagen = ImageDataGenerator(rescale=1./255)#defining validation dataset

PREPROCESSING OF TRAINNING DATASET

In [11]:
train_generator = train_datagen.flow_from_directory(
    'C:/Users/Ayan/OneDrive/Desktop/dataset/training_set', #always put forward slashes rather than backward cause python don support it
    target_size=(125, 125),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


PREPROCESSING OF VALIDATION DATASET

In [13]:
val_generator = val_datagen.flow_from_directory(
    'C:/Users/Ayan/OneDrive/Desktop/dataset/test_set',
    target_size=(125, 125),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


PREPROCESSING THE TEST SET

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'C:/Users/Ayan/OneDrive/Desktop/dataset/test_set',
    target_size=(125, 125),
    batch_size=32,
    class_mode='binary',        # or 'categorical' if multiclass
    shuffle=False               # VERY IMPORTANT for predictions & metrics!
)

Found 2000 images belonging to 2 classes.


PART-2:- BUILDING THE CNN

INITIALIZE THE CNN

In [19]:
cnn = tf.keras.models.Sequential()

CONVOLUTIONAL LAYER

In [22]:
cnn.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    strides = 1,
    activation = 'relu',
    input_shape = [125,125,3]
))#use to detect pattern in 2d image for my model

POOLING

In [25]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides = 2))

ADDING NEXT CONVOLUTIONAL LAYERS

In [27]:
cnn.add(tf.keras.layers.Conv2D(filters = 64,kernel_size = 3,strides = 1,activation = 'relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides = 2))
cnn.add(tf.keras.layers.Conv2D(filters = 64,kernel_size = 3,strides = 1,activation = 'relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides = 2))

FLATTENING

In [30]:
cnn.add(tf.keras.layers.Flatten())

FULLY CONNECTED LAYERS OR HIDDEN LAYERS

In [33]:
cnn.add(tf.keras.layers.Dense(units = 128,activation = 'relu'))

OUTPUT LAYER

In [37]:
cnn.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid'))

PART-3:- TRAINNING THE CNN MODEL

COMPILE THE CNN

In [41]:
cnn.compile(optimizer ='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 61, 61, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 59, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 29, 29, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 64)        0

TRAIN CNN

In [51]:
cnn.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
250/250 [==============================] - 360s 1s/step - loss: 0.4519 - accuracy: 0.7846 - val_loss: 0.4236 - val_accuracy: 0.8055
Epoch 2/20
250/250 [==============================] - 351s 1s/step - loss: 0.4188 - accuracy: 0.8060 - val_loss: 0.3959 - val_accuracy: 0.8220
Epoch 3/20
250/250 [==============================] - 360s 1s/step - loss: 0.4245 - accuracy: 0.8076 - val_loss: 0.4076 - val_accuracy: 0.8135
Epoch 4/20
250/250 [==============================] - 360s 1s/step - loss: 0.4032 - accuracy: 0.8126 - val_loss: 0.3900 - val_accuracy: 0.8255
Epoch 5/20
250/250 [==============================] - 367s 1s/step - loss: 0.3964 - accuracy: 0.8188 - val_loss: 0.3743 - val_accuracy: 0.8305
Epoch 6/20
250/250 [==============================] - 368s 1s/step - loss: 0.3878 - accuracy: 0.8249 - val_loss: 0.3623 - val_accuracy: 0.8410
Epoch 7/20
250/250 [==============================] - 368s 1s/step - loss: 0.3759 - accuracy: 0.8280 - val_loss: 0.3466 - val_accuracy: 0.8525

In [78]:
from tensorflow.keras.preprocessing import image
import numpy as np
img_path = 'Downloads/photo-1664822132578-5efe9340d5ee.jpg'
img = image.load_img(img_path, target_size=(125, 125))  # Same size as training data

# Convert image to array and preprocess
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize to [0,1] like training data

# Make the prediction
prediction = cnn.predict(img_array)
probability = prediction*100
print(train_generator.class_indices) #print the what value the model assigned for cat and dog
if prediction[0][0] >= 0.5:
    print("I am {probability}It's a Dog")
else:
    print("It's a Cat")

1/1 [==============================] - 0s 320ms/step
{'cats': 0, 'dogs': 1}
[[0.93167675]]
It's a Dog
